In [1]:
#Import Dependencies and variables
import os
import plaid
import requests
import datetime
import json
import pandas as pd
from pandas.io.json import json_normalize
%matplotlib inline

In [2]:
def pretty_print_response(response):
  print(json.dumps(response, indent=4, sort_keys=True))

In [3]:
PLAID_CLIENT_ID = os.getenv('PLAID_CLIENT_ID')
PLAID_SBX_SECRET_KEY = os.getenv('PLAID_SBX_SECRET_KEY')
PLAID_PUBLIC_KEY = os.getenv('PLAID_PUBLIC_KEY')
PLAID_ENV = os.getenv('PLAID_ENV', 'sandbox')
PLAID_PRODUCTS = os.getenv('PLAID_PRODUCTS', 'transactions')

In [4]:
print(len(PLAID_CLIENT_ID))
print(len(PLAID_PUBLIC_KEY))
print(len(PLAID_SBX_SECRET_KEY))

24
30
30


In [5]:
# Create client object
client = plaid.Client(client_id=PLAID_CLIENT_ID, secret=PLAID_SBX_SECRET_KEY, public_key=PLAID_PUBLIC_KEY, environment= 'sandbox')

In [6]:
#select an institution for processing
INSTITUTION_ID = "ins_109508"

# Create public token to be exchanged for institution access token
create_tkn_response = client.Sandbox.public_token.create(INSTITUTION_ID, ['transactions','income','assets'])

# Exchange public token for access token
exchange_response = client.Item.public_token.exchange(create_tkn_response['public_token'])

# Store access token as variable
access_token = exchange_response['access_token']


In [7]:
# Test the access token by requesting and printing the available test accounts
account_response = client.Accounts.get(access_token)
account_response

{'accounts': [{'account_id': 'No5oBzn3VrSeklv4qPkKhedR7vXDpLcWDg9zR',
   'balances': {'available': 100,
    'current': 110,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '0000',
   'name': 'Plaid Checking',
   'official_name': 'Plaid Gold Standard 0% Interest Checking',
   'subtype': 'checking',
   'type': 'depository'},
  {'account_id': 'P858x3EdRBIox9BP7MxJh1JQl8XjpWH7M9gn3',
   'balances': {'available': 200,
    'current': 210,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '1111',
   'name': 'Plaid Saving',
   'official_name': 'Plaid Silver Standard 0.1% Interest Saving',
   'subtype': 'savings',
   'type': 'depository'},
  {'account_id': 'jjlj9ygK38UGzpjgXDz5H6XRy7jPAzt1XBDkJ',
   'balances': {'available': None,
    'current': 1000,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '2222',
   'name': 'Plaid CD',
   'officia

In [8]:
account_response.keys()

dict_keys(['accounts', 'item', 'request_id'])

# Account Transactions with Plaid

In this section, you will use the Plaid Python SDK to connect to the Developer Sandbox account and grab a list of transactions. You will need to complete the following steps:


1. Use the access token to fetch the transactions for the last 90 days
2. Print the categories for each transaction type
3. Create a new DataFrame using the following fields from the JSON transaction data: `date, name, amount, category`. (For categories with more than one label, just use the first category label in the list)
4. Convert the data types to the appropriate types (i.e. datetimeindex for the date and float for the amount)

In [9]:
# Get transactions for institution for specific date range
start_date = '{:%Y-%m-%d}'.format(datetime.datetime.now() + datetime.timedelta(-90))
end_date = '{:%Y-%m-%d}'.format(datetime.datetime.now())

# Use the access token to fetch the transactions for the last 90 days
transaction_response = client.Transactions.get(access_token,start_date,end_date)

# Print JSON output
print(json.dumps(transaction_response['transactions'][:2],indent=4, sort_keys=True))
transaction_response.keys()

[
    {
        "account_id": "P858x3EdRBIox9BP7MxJh1JQl8XjpWH7M9gn3",
        "account_owner": null,
        "amount": 25,
        "category": [
            "Payment",
            "Credit Card"
        ],
        "category_id": "16001000",
        "date": "2019-09-18",
        "iso_currency_code": "USD",
        "location": {
            "address": null,
            "city": null,
            "country": null,
            "lat": null,
            "lon": null,
            "postal_code": null,
            "region": null,
            "store_number": null
        },
        "name": "CREDIT CARD 3333 PAYMENT *//",
        "payment_meta": {
            "by_order_of": null,
            "payee": null,
            "payer": null,
            "payment_method": null,
            "payment_processor": null,
            "ppd_id": null,
            "reason": null,
            "reference_number": null
        },
        "pending": false,
        "pending_transaction_id": null,
        "transaction_id": 

dict_keys(['accounts', 'item', 'request_id', 'total_transactions', 'transactions'])

In [10]:
# Print the categories for each transaction
category_info = client.Transactions.get(access_token,start_date,end_date)
#if transactions in category_info ["transactions"]:
    #if transactions ['name'] == 
#category_info['transactions'][0].key()
print({category
       for transaction in category_info['transactions'] if transaction['category'] is not None
       for category in transaction['category']})


{'Car Service', 'Restaurants', 'Sporting Goods', 'Ride Share', 'Debit', 'Transfer', 'Credit', 'Shops', 'Recreation', 'Payment', 'Food and Drink', 'Credit Card', 'Gyms and Fitness Centers', 'Deposit', 'Airlines and Aviation Services', 'Travel', 'Coffee Shop'}


In [49]:
# Create a new DataFrame using the following fields from the JSON transaction data: date, name, amount, category.
tracks_df = json_normalize(transaction_response, ['transactions'])
filtered_df = tracks_df[['date', 'amount', 'name', 'category']],index_col="date", infer_datetime_format=True
filtered_df

SyntaxError: can't assign to literal (<ipython-input-49-878defc14f27>, line 3)

### 4. Convert the data types to the appropriate types 

(i.e. datetimeindex for the date and float for the amount)

In [46]:
filtered_df.set_index(filtered_df['date'], inplace=True)


In [47]:
filtered_df.sort_index(inplace=True)
filtered_df

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,date,amount,name,category
date,,,,
2019-08-28,2019-08-28,2078.50,AUTOMATIC PAYMENT - THANK,[Payment]
2019-08-28,2019-08-28,500.00,KFC,"[Food and Drink, Restaurants]"
2019-08-28,2019-08-28,500.00,Madison Bicycle Shop,"[Shops, Sporting Goods]"
2019-08-29,2019-08-29,500.00,Tectra Inc,"[Food and Drink, Restaurants]"
2019-09-01,2019-09-01,6.33,Uber 072515 SF**POOL**,"[Travel, Car Service, Ride Share]"
2019-09-03,2019-09-03,500.00,United Airlines,"[Travel, Airlines and Aviation Services]"
2019-09-13,2019-09-13,-4.22,INTRST PYMNT,"[Transfer, Credit]"
2019-09-14,2019-09-14,89.40,SparkFun,"[Food and Drink, Restaurants]"
2019-09-15,2019-09-15,12.00,McDonald's,"[Food and Drink, Restaurants]"


---

# Income Analysis with Plaid

In this section, you will use the Plaid Sandbox to complete the following:
1. Determine the previous year's gross income and print the results
2. Determine the current monthly income and print the results
3. Determine the projected yearly income and print the results

In [17]:
# YOUR CODE HERE

---

# Budget Analysis
In this section, you will use the transactions DataFrame to analyze the customer's budget

1. Calculate the total spending per category and print the results (Hint: groupby or count transactions per category)
2. Generate a bar chart with the number of transactions for each category 
3. Calulate the expenses per month
4. Plot the total expenses per month

### Calculate the expenses per category

In [18]:
# YOUR CODE HERE

### Calculate the expenses per month

In [19]:
# YOUR CODE HERE